# Arquivos especiais - Excel

Em lição anterior vimos [como manipular arquivos texto](https://neci-python.blogspot.com.br/p/blog-page_27.html).

Para melhor aproveitar esta lição é preciso conhecer [dicionarios](https://neci-python.blogspot.com.br/p/blog-page_99.html).

Agora vamos ver como manipular arquivos especiais de diversos tipos, começando com arquivos **.xls** e **.xlsx** com workbooks e planilhas Excel.

Como no caso de arquivos texto, a manipulação de arquivos Excel segue a sequência genérica de passos:
- abrir o arquivo
- acessar o seu conteúdo, no caso acessar a planilha
- fechar o arquivo

A diferença está nos detalhes de como fazer estes passos. Veremos que lidar com planilhas requer um procedimento mais complicado do que lidar com textos.

### Planilhas do Excel

Quando manipulamos arquivos texto o modêlo conceitual era simples: um string de caracteres e símbolos. Em contraste para manipular planilhas temos que ter em mente um modêlo bem mais complicado. Num modêlo símples uma planilha é uma grade formada por linhas (**rows**) e colunas (**columns**, ou **cols**) em cuja intersecção ficam células (**cells**). 


![alt text](https://dl.dropboxusercontent.com/u/50004393/book1small.png)

Linhas são ordenadas por numeros sequenciais, como na figura acima. Colunas são identificadas por letras (alfabeto inglês) na seguinte ordem:

A, B, C, D, ... X, Y, Z, 

AA, AB, AC, ... AX, AY, AZ, 

BA, BB, BC, ...

...

Células contém valores (**value**) que podem ser números, strings e outras coisas. Toda célula possue uma identificação (endereço) que é a sua coluna e sua linha. 

Por exemplo, na figura acima a célula com endereço **A2** está na coluna **A** interseção com linha **2**, e contém o string 'ABEL MESQUITA JR.'.

Já a celula em **C8** contém o string 'MG'.

Para lêr o valor em uma célula, ou escrever um valor numa célula, vamos precisar do seu endereço.

As planilhas propriamente ditas são chamadas de **worksheets** (ou **sheets** para simplificar) e o arquivo Excel guarda uma espécie de pasta de planilhas, chamado **workbook**. Cada **sheet** do workbook tem um *nome* e para referenciar um dado **sheet** é necessário saber o seu *nome*.

---

A manipulação de arquivos Excel será feita através de módulos especializados. Já vimos um módulo destes na lição [*listas de deputados e seus partidos*](https://neci-python.blogspot.com.br/p/blog-page_21.html), o módulo **xlrd** que permite leitura de planilhas.

Nesta lição vamos usar o módulo **openpyxl** que é usado em nosso livro referência [Al Sweigart, "Automate the Boring Stuff With Python"](http://www.amazon.com/Automate-Boring-Stuff-Python-Programming/dp/1593275994/ref=sr_1_1?s=books&ie=UTF8&qid=1462158743&sr=1-1&keywords=automate+the+boring+stuff+with+python). Este poderoso módulo permite abrir, ler, criar e editar planilhas Excel.

---

Vimos que a função **open()** retorna um objeto **File**, uma espécie de "cano" entre nosso programa e o arquivo. No caso de arquivos texto o objeto **File** tinha métodos para a leitura fácil do texto em forma de um string ou uma lista de strings representando frases.

No caso de arquivos Excel não é tão simples. O conteúdo do arquivo representa um **workbook** contendo um determinado número de **worksheets** (planilhas). Então ao abrir um arquivo destes vamos acessar toda uma coleção de objetos que compõem o **workbook**. Estes objetos estão organizados numa hierarquia em que objetos contem objetos subalternos e mostrada abaixo de modo simplificado:

![alt text](https://dl.dropboxusercontent.com/u/50004393/excel.png)

É bom frisar que estes são apens uns poucos das *muitas dezenas* de objetos que compõem um workbook Excel. Felizmente na prática não precisaremos de tantos para manipulação simples de planilhas. Em todo o caso sempre pode-se consultar a documentação do módulo usado (como o openpyxl).

---

Então o que fazer com todos estes objetos? Bem, vejamos o caso mais simples de leitura de uma planilha num workbook e exame do conteúdo de suas células.




### Lendo conteúdo de células de uma planilha

Seguindo as setas na figura acima podemos esboçar a sequencia de passos para chegar até o conteúdo de células de uma planilha:

- abrir o arquivo xlsx na forma permitida pelo módulo openpyxl
- obter o objeto **workbook**
- usando o nome da planilha desejada, obter o objeto **worksheet** correspondente
- usando endereços (p.ex. **A5**) chegar até células de interesse
- obter o conteúdo da célula de interesse lendo seu valor (**value**)

Vejamos os detalhes de como fazer tudo isto. Como objeto de estudo vamos usar a planilha 'Sheet 1' do workbook **dep1.xlsx** [que pode ser baixado daqui](https://dl.dropboxusercontent.com/u/50004393/lixo/dep1.xlsx) e que está mostrada numa figura acima.




In [ ]:
import openpyxl, os

# vamos abrir o arquivo xlsx obtendo um objeto workbook correspondente
wb = openpyxl.load_workbook('dep1.xlsx')

In [ ]:
# podemos listar todos nomes de planilhas neste workbook
folhas = wb.get_sheet_names()
folhas

In [ ]:
# a planilha de interesse chama-se Sheet 1, vamos obter seu objeto
sheet = wb.get_sheet_by_name('Sheet1')

# podemos contar o numero de linhas e de colunas desta planilha
nr = sheet.get_highest_row()
nc = sheet.get_highest_column()
print('rows = ', nr)
print('columns = ',nc)

Examinando a figura acima que mostra a planilha em questão vemos que de fato existem 10 linhas (mas cuidado, a primeira é uma linha de títulos de colunas!) e vê-se que há mais de cinco colunas (são 18 mas não caberiam na figura).

Nosso módulo **openpyxl** permite acessar uma célula diretamente do objeto **worksheet** se indexarmos o mesmo com o endereço no formato tipo **B7** (letra da coluna seguido do número da linha). Por exemplo a célula na linha 3 e coluna B pode ser acessada pela expressão **sheet[B3]** e seu conteúdo (valor) por:

```python
sheet[B3].value
```

Vamos então listar o conteúdo das células de nossa planilha exemplo, imprimindo as 4 primeiras colunas (A,B,C,D) linha por linha. Para tanto vamos iterar por todas linhas (menos a primeira que só contem títulos) criando o endereço de cada célula como mostrado abaixo (lembre-se que a função **str()** converte um número em um string):

In [ ]:
for rw in range(2,nr + 1):
    nome = sheet['A' + str(rw)].value
    partido = sheet['B' + str(rw)].value
    uf = sheet['C' + str(rw)].value
    status = sheet['D' + str(rw)].value
    print(nome,partido,uf,status)
    

Caso fosse interessante poderiamos colocar estes dados num dicionário, onde a chave de cada item é o nome (tomado na coluna A) e o valor é uma tupla com os conteúdos das colunas B,C,D, (partido,uf,status):

In [ ]:
dic = {}  # cria dicionario vazio
for rw in range(2,nr + 1):
    nome = sheet['A' + str(rw)].value
    partido = sheet['B' + str(rw)].value
    uf = sheet['C' + str(rw)].value
    status = sheet['D' + str(rw)].value
    valor = (partido,uf,status)
    dic[nome] = valor

In [ ]:
# podemos imprimir bonitinho o dicionario usando o modulo pprint
import pprint
pprint.pprint(dic)

### Salvando dados numa planilha Excel

Ocasionalmente temos a tarefa reversa, isto é, temos uma estrutura de dados como por exemplo um dicionário ou o que quer que seja e gostariamos de guardar estes dados numa planilha Excel.

Vejamos como fazê-lo usando nosso módulo **openpyxl**.

Para exemplificar usaremos nosso dicionário *dic* que está criado e salvaremos o mesmo numa nova planilha chamada 'camara.xlsx'.

A sequência de passos é a seguinte:

- criar um novo workbook (já vem com um sheet, vazio, chamado 'Sheet')
- renomear o worksheet default com o nome desejado
- escrever dados nas células da planilha
- salvar o workbook num arquivo xlsx



In [ ]:
# cria um novo workbook contendo um worksheet vazio
wb = openpyxl.Workbook()

# vejamos os nomes de planilhas criadas
wb.get_sheet_names()


In [ ]:
# mudemos o nome para Lista de deputados
sheet = wb.get_sheet_by_name('Sheet')
print(sheet.title)
sheet.title = 'Lista de deputados'
sheet.title

Vamos percorrer as chaves do dicionário *dic* e para cada chave (que é o *nome* de um deputado) pegamos seu valor correspondente. Do valor obtemos o *partido*, a *uf* e o *status*. 

Colocamos *nome, partido, uf, status* nesta ordem nas células de uma linha e passamos para a chave seguinte.

In [ ]:
# percorre todos items do dicionario
rw = 1
for key,val in dic.items():
    nome = key
    partido = val[0]
    uf = val[1]
    status = val[2]
    
    # preenche as células na linha de indice rw
    lin = str(rw)
    sheet['A' + lin] = nome
    sheet['B' + lin] = partido
    sheet['C' + lin] = uf
    sheet['D' + lin] = status
    
    # proxima linha
    rw = rw + 1


OK preenchemos a planilha com os dados de nosso dicionario. Agora precisamos salvar o arquivo Excel correspondente. Isto é feito pelo método **save(nome)** de objetos **workbook**

In [ ]:
wb.save('camara.xlsx')

Se tudo foi bem deve ter aparecido um arquivo *camara.xlsx* no diretório corrente. Abrindo este arquivo com o Excel deve aparecer algo assim:


![alt text](https://dl.dropboxusercontent.com/u/50004393/lixo/camara.PNG)



Existe uma série de outras coisas que podemos manipular numa planilha usando nosso módulo **openpyxl**. Por exemplo:
- modificar fontes de texto, largura e altura de colunas
- mudar cores de fundo de células
- inserir **formulas** em células, no padrão do Excel
- criar e inserir gráficos na planilha
- etc

Entretanto para nossos fins neste curso basta o já visto. Se você tem interesse consulte o livro referência para detalhes.
